In [18]:
# https://www.bitaddress.org/
# https://en.wikipedia.org/wiki/Base58

BASE58_CHARS = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'


def map_base58():
    '''Create a map of chars for Base58'''
    tmp_dir = {}
    i = 0
    for c in BASE58_CHARS:
        tmp_dir[c] = i
        i = i + 1
    return tmp_dir


BASE58_MAP = map_base58()


def base58_enc(in_num):
    '''Encode a number into Bitcoin Base58 format'''
    if in_num < 0:
        raise Exception("Positive integers only")
    tmp_out = []
    while in_num > 0:
        tmp_out.append(BASE58_CHARS[in_num % 58])
        in_num = in_num // 58
    tmp_out.reverse()
    return ''.join(tmp_out)


def base58_dec(in_str):
    '''Decode a Bitcoin Base58 string to a number'''
    out_num = 0
    for c in in_str:
        if c not in BASE58_MAP:
            raise Exception("Invalid char, not in Base58: %c" % c)
        out_num = out_num*58 + BASE58_MAP[c]
    return out_num

In [19]:
# For testing
PRIV_KEY_BASE58 = 'L3BQRZyUzNPUPbt1HtGby9UwVb5iz2RyEk9jQk1vqhnL5CwFZHiX'
BITCOIN_ADDR_BASE58 = '1D2Gme2513ncWsxB4DchzT3ukeNUXYVv3c'

print(base58_enc(123))
print(base58_dec('38'))
print(base58_dec(PRIV_KEY_BASE58))
print(base58_dec(BITCOIN_ADDR_BASE58))

38
123
16384730769480552998872844202963833909082627022173914913578359054339044055507470509937806088
3233478352685485411235947024398804356817507813717140601691
